In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import truncnorm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import GridSearchCV, PredefinedSplit

from pysim import SimBoostRegressor, SimBoostClassifier

## Regression

In [2]:
random_state = 0
np.random.seed(random_state)
# data generation
beta1 = np.array([.2, 0.3, 0.5, 0, 0, 0, 0, 0, 0, 0])
beta2 = np.array([0, .2, 0.3, 0.5, 0, 0, 0, 0, 0, 0])
beta3 = np.array([0, 0, 0.2, 0.3, 0.5, 0, 0, 0, 0, 0])

beta = np.vstack([beta1, beta2, beta3])
model_list = [lambda x: 0.2 * np.exp(-4 * x), lambda x: 3 * x ** 2, lambda x: 2.5 * np.sin(1.5 * np.pi * x)]

x = truncnorm.rvs(a=-3, b=3, loc = 0, scale=1 / 3, size=(20000, 10), random_state=random_state)
noise = np.random.randn(20000).reshape(-1, 1)
y = np.reshape(0.2 * np.exp(-4 * np.dot(x, beta1)) + \
               3 * (np.dot(x, beta2)) ** 2 + 2.5 * np.sin(np.pi * 1.5 * np.dot(x, beta3)), [-1, 1]) + noise
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=random_state)

In [3]:
%%time 

middle_update = {"method":"bfgs",
                 "tol":0.0001,
                 "max_middle_iter":3,
                 "n_middle_iter_no_change":3, 
                 "max_inner_iter":20,
                 "learning_rate":1e-3,
                 "stratify":True,
                 "verbose":False}
        
clf = SimBoostRegressor(n_estimators=20, knot_num=10, knot_dist="uniform", spline="smoothing_spline", learning_rate=1,
                        reg_lambda=[0.1, 0.2, 0.3, 0.4, 0.5], degree=3,
                        reg_gamma="GCV", middle_update=middle_update, meta_info=None)
clf.fit(train_x, train_y)

AttributeError: 'SimBoostRegressor' object has no attribute 'inner_update'

In [4]:
clf.visualize()

NotFittedError: This SimBoostRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
clf.validation_performance()

In [ ]:
mean_squared_error(test_y, clf.predict(test_x))

## LogitBoost

In [ ]:
random_state = 0
np.random.seed(random_state)
# data generation
beta1 = np.array([.2, 0.3, 0.5, 0, 0, 0, 0, 0, 0, 0])
beta2 = np.array([0, .2, 0.3, 0.5, 0, 0, 0, 0, 0, 0])
beta3 = np.array([0, 0, 0.2, 0.3, 0.5, 0, 0, 0, 0, 0])
beta = np.vstack([beta1, beta2, beta3])

x = truncnorm.rvs(a=-3, b=3, loc = 0, scale=1 / 3, size=(10000, 10), random_state=random_state)
y = np.reshape(0.2 * np.exp(-4 * np.dot(x, beta1)) + \
               3 * (np.dot(x, beta2)) ** 2 + 2.5 * np.sin(np.pi * 1.5 * np.dot(x, beta3)), [-1, 1]) + \
    np.random.randn(10000).reshape(-1, 1)
y[y <= 0] = 0
y[y > 0] = 1
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=random_state)

In [ ]:
%%time 

middle_update = {"method":"bfgs",
                 "tol":0.0001,
                 "max_middle_iter":3,
                 "n_middle_iter_no_change":3, 
                 "max_inner_iter":20,
                 "learning_rate":1e-3,
                 "stratify":True,
                 "verbose":False}

clf = SimBoostClassifier(n_estimators=10, knot_num=10, knot_dist="quantile", spline="smoothing_spline", learning_rate=1,
                         reg_lambda=[0.1, 0.2, 0.3, 0.4, 0.5], degree=3, 
                         reg_gamma="GCV", middle_update=middle_update, meta_info=None)
clf.fit(train_x, train_y)
clf.visualize()

In [ ]:
clf.validation_performance()

In [ ]:
roc_auc_score(test_y, clf.predict_proba(test_x))